In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv("../input/epitope-prediction/input_bcell.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.countplot(x = "target" ,data = df)

In [ ]:
df["protein_seq_len"] = df["protein_seq"].apply(len)

In [ ]:
df["peptide_seq_len"] = df["peptide_seq"].apply(len)

In [ ]:
df.head()

In [ ]:
df = pd.get_dummies(df,columns = ["target"],drop_first=True)

In [ ]:
df.head()

In [ ]:
df["parent_protein_id"].nunique()

In [ ]:
df["start_position"].nunique()

In [ ]:
plt.figure(figsize=(18,12))

sns.heatmap(df.corr(),linewidth = 1.5, annot = True)

In [ ]:
df.drop(columns = ["peptide_seq"], axis =1 ,inplace =True)

In [ ]:
df.head()

In [ ]:
df[df["parent_protein_id"] == "A2T3T0"].head()

In [ ]:
df.drop(columns = ["protein_seq"], axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
df = pd.get_dummies(df,columns=["parent_protein_id"],drop_first = True)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(["target_1"], axis =1).values
y = df["target_1"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
early_stop=EarlyStopping(monitor="val_loss",mode="min",patience=25,verbose=1)

In [ ]:
model = Sequential()

# Choose whatever number of layers/neurons you want.
model.add(Dense(1024,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.5))
#Binary Classification
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy",optimizer="adam")

In [ ]:
model.fit(X_train,y_train,epochs=150,validation_data=(X_test,y_test),batch_size=256,callbacks=[early_stop])

In [ ]:
history=pd.DataFrame(model.history.history)


In [ ]:
history.plot()

In [ ]:
pred=model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test,pred))

In [ ]:
print(confusion_matrix(y_test,pred))